In [2]:
from data.data_utils import get_distillation_dataloader

In [3]:
from configs import dataset as DATA_CONFIG
from configs import fsdp_config as FSDP_CONFIG
from configs import train_config as TRAIN_CONFIG
from configs import distillation_config as DISTIL_CONFIG

In [4]:
train_config, fsdp_config, distil_config, data_config = TRAIN_CONFIG(), FSDP_CONFIG(), DISTIL_CONFIG(), DATA_CONFIG()


In [ ]:
args_dict = {'model_name': 'EleutherAI/pythia-410m-deduped', 'dataset.file': 'datasets/loader/squad.py', 'lr': 1e-06, 'num_epochs': 5, 'batch_size_training': 4, 'val_batch_size': 4, 'output_dir': 'train/output/path', 'distillation_config.model_name': 'meta-llama/Llama-2-7b-chat-hf', 'distillation': True, 'distillation_config.enable_fsdp': True, 'distillation_config.pure_bf16': True, 'distillation_config.distil_factor': 1.5, 'save_step': 100}

In [ ]:
train_dataloader, teacher_train_dataloader, eval_dataloader, teacher_eval_dataloader = get_distillation_dataloader(data_config, train_config, distil_config, student_tokenizer, teacher_tokenizer, rank)